<a href="https://colab.research.google.com/github/medha-sagar/CSE547-BigDataForMachineLearning/blob/master/Copy_of_CSE547_Colab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 547 - Colab 1
## Wordcount in Spark

Adapted From Stanford CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 59kB/s 
     |████████████████████████████████| 204kB 48.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=083078ce7a69a4f875cf6dce0bb6e4ec78cc2a0eaae62a6e009e5015e112529b
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1SE6k_0YukzGd5wK-E4i6mG83nydlfvSa'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pg100.txt')

If you executed the cells above, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

### Your task

If you run successfully the setup stage, you are ready to work on the *pg100.txt* file which contains a copy of the complete works of Shakespeare.

Write a Spark application which outputs the number of words that start with each letter. This means that for every letter we want to count the total number of (non-unique) words that start with a specific letter. In your implementation **ignore the letter case**, i.e., consider all words as lower case. Also, you can ignore all the words **starting** with a non-alphabetic character.

For this task we ask you to the [**RDD MapReduce API**](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) from spark (map, reduceByKey, flatMap, etc.) instead of **DataFrame API**. 

In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [8]:
spark

In [0]:
Shakespeare_Document = sc.textFile("pg100.txt")
words = Shakespeare_Document.flatMap(lambda line: line.split(" "))
words_new = words.filter(lambda x: x != "")

In [70]:
words_new.take(16)

['The',
 'Project',
 'Gutenberg',
 'EBook',
 'of',
 'The',
 'Complete',
 'Works',
 'of',
 'William',
 'Shakespeare,',
 'by',
 'William',
 'Shakespeare',
 'This',
 'eBook']

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!

In [0]:
wordCounts = words_new.map(lambda word: (word[0].lower(), 1)) #.reduceByKey(lambda a,b:a +b)

In [74]:
wordCounts.take(16)

[('t', 1),
 ('p', 1),
 ('g', 1),
 ('e', 1),
 ('o', 1),
 ('t', 1),
 ('c', 1),
 ('w', 1),
 ('o', 1),
 ('w', 1),
 ('s', 1),
 ('b', 1),
 ('w', 1),
 ('s', 1),
 ('t', 1),
 ('e', 1)]

In [0]:
letterCount = wordCounts.reduceByKey(lambda a, b: a+b).sortBy(lambda r: -r[1])

In [77]:
letterCount.take(100)

[('t', 123602),
 ('a', 84836),
 ('s', 65705),
 ('i', 62167),
 ('h', 60563),
 ('w', 59597),
 ('m', 55676),
 ('b', 45455),
 ('o', 43494),
 ('f', 36814),
 ('c', 34567),
 ('d', 29713),
 ('l', 29569),
 ('p', 27759),
 ('n', 26759),
 ('y', 25855),
 ('g', 20782),
 ('e', 18697),
 ('r', 14265),
 ('k', 9418),
 ('u', 9170),
 ('v', 5728),
 ("'", 3804),
 ('j', 3339),
 ('q', 2377),
 ('[', 2073),
 ('(', 639),
 ('1', 458),
 ('"', 356),
 ('<', 248),
 ('2', 95),
 ('z', 71),
 ('3', 59),
 ('.', 52),
 ('-', 52),
 ('4', 46),
 ('5', 35),
 ('9', 28),
 ('*', 24),
 ('6', 22),
 ('&', 21),
 ('7', 17),
 ('8', 15),
 ('x', 14),
 (']', 7),
 ('0', 6),
 ('#', 3),
 ('?', 2),
 ('}', 2),
 ('/', 2),
 ('_', 1),
 ('$', 1),
 (':', 1)]